In [1]:
from data_extractor import DataExtractor

In [2]:
import h3

h3.__version__

'4.2.2'

In [3]:
extractor = DataExtractor()
# extractor.download_catalog(["precip"])

In [4]:
precip, tmin, tmax = extractor.process_catalog(["precip", "tmin", "tmax"])

In [5]:
_, state_polygons = extractor.aggregate_shape(precip)

c:\Monash\FIT3164\github_repo\projenv\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [6]:
# _, state_polygons = extractor.aggregate_shape(precip)
precip_gdf_hex, hexagon_polygons = extractor.aggregate_h3_australia(precip, state_polygons, resolution=4)
tmax_gdf_hex, _ = extractor.aggregate_h3_australia(tmax, state_polygons)
tmin_gdf_hex, _ = extractor.aggregate_h3_australia(tmin, state_polygons)

c:\Monash\FIT3164\github_repo\2025_DataScienceProject2_DS07\data_gen\data_extractor.py:131: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:7844

  aggregated_gdf = gpd.sjoin(
c:\Monash\FIT3164\github_repo\2025_DataScienceProject2_DS07\data_gen\data_extractor.py:131: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:7844

  aggregated_gdf = gpd.sjoin(
c:\Monash\FIT3164\github_repo\2025_DataScienceProject2_DS07\data_gen\data_extractor.py:131: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPS

In [7]:
precip_gdf, _ = extractor.aggregate_shape(precip)
tmax_gdf, _ = extractor.aggregate_shape(tmax)
tmin_gdf, _ = extractor.aggregate_shape(tmin)

c:\Monash\FIT3164\github_repo\projenv\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
c:\Monash\FIT3164\github_repo\projenv\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
c:\Monash\FIT3164\github_repo\projenv\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [8]:
state_polygons.to_csv("../../data/processed/processed_climate_data/suburb/suburb_geometries.csv")

In [9]:
hexagon_polygons.to_csv("../../data/processed/processed_climate_data/hexagon/hexagon_geometries.csv")

In [10]:
import pandas as pd
import re

def refactor_columns(df, metric):
    # Define month → season mapping
    month_to_season = {
        1: 'Summer', 2: 'Summer', 12: 'Summer',
        3: 'Autumn', 4: 'Autumn', 5: 'Autumn',
        6: 'Winter', 7: 'Winter', 8: 'Winter',
        9: 'Spring', 10: 'Spring', 11: 'Spring'
    }

    # Regex pattern for matching column names
    pattern = re.compile(r'(\d{4})_month_(\d{1,2})')

    # Separate month and non-month columns
    non_month_cols = []
    season_groups = {}

    for col in df.columns:
        match = pattern.fullmatch(col)
        if match:
            year, month = int(match.group(1)), int(match.group(2))
            season = month_to_season[month]

            # Adjust year for December, as it belongs to next year's Summer
            if month == 12:
                year += 1

            season_key = f"{metric}_{year}_{season}"
            season_groups.setdefault(season_key, []).append(col)
        else:
            non_month_cols.append(col)

    # Compute seasonal averages
    seasonal_df = pd.DataFrame(index=df.index)

    for season_col, month_cols in season_groups.items():
        seasonal_df[season_col] = df[month_cols].mean(axis=1)
    # Combine with original non-month columns
    final_df = pd.concat([df[non_month_cols], seasonal_df], axis=1)

    return final_df

precip_gdf = refactor_columns(precip_gdf, "precip")
tmin_gdf = refactor_columns(tmin_gdf, "tmin")
tmax_gdf = refactor_columns(tmax_gdf, "tmax")

precip_gdf_hex = refactor_columns(precip_gdf_hex, "precip")
tmin_gdf_hex = refactor_columns(tmin_gdf_hex, "tmin")
tmax_gdf_hex = refactor_columns(tmax_gdf_hex, "tmax")

In [11]:
precip_gdf.to_csv("../../data/processed/processed_climate_data/suburb/seasonal_mean_precip_mm_suburb.csv")
tmin_gdf.to_csv("../../data/processed/processed_climate_data/suburb/seasonal_mean_tmin_suburb.csv")
tmax_gdf.to_csv("../../data/processed/processed_climate_data/suburb/seasonal_mean_tmax_suburb.csv")

In [14]:
tmin_gdf

,SAL_NAME21,STE_NAME21,tmin_2000_Summer,tmin_2000_Autumn,tmin_2000_Winter,tmin_2000_Spring,tmin_2001_Summer,tmin_2001_Autumn,tmin_2001_Winter,tmin_2001_Spring,...,tmin_2018_Summer,tmin_2018_Autumn,tmin_2018_Winter,tmin_2018_Spring,tmin_2019_Summer,tmin_2019_Autumn,tmin_2019_Winter,tmin_2019_Spring,tmin_2020_Summer,tmin_2020_Autumn
0,ACT Remainder - Belconnen,Australian Capital Territory,13.343506,8.730469,1.570312,8.276530,14.729004,7.463705,2.180339,7.075033,...,14.657715,8.602051,1.592122,7.511881,15.755859,8.925293,1.708659,6.923340,15.014160,8.233724
1,ACT Remainder - Booth,Australian Capital Territory,10.915136,5.889128,-1.058008,5.958203,12.188931,4.785872,-0.259310,4.826953,...,12.191602,5.849283,-0.881315,5.148112,13.215039,6.193295,-0.890625,4.641732,12.398242,5.483268
2,ACT Remainder - Canberra Central,Australian Capital Territory,12.561523,7.779297,0.649089,7.517252,13.925781,6.552734,1.305990,6.340820,...,13.892578,7.706055,0.755534,6.773763,14.942383,8.009114,0.865560,6.194010,14.229492,7.349609
3,ACT Remainder - Coree,Australian Capital Territory,12.023274,7.530056,0.674913,7.075304,13.408093,6.255642,1.271701,5.866320,...,13.313586,7.373372,0.643772,6.282118,14.444661,7.722439,0.740668,5.708550,13.661241,6.984375
4,ACT Remainder - Cotter River,Australian Capital Territory,9.837820,5.254302,-1.131580,5.040737,11.187430,4.029296,-0.481166,3.842727,...,11.096424,5.111212,-1.135138,4.198126,12.229050,5.476237,-1.090820,3.672456,11.391183,4.705753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15348,Zillmere,Queensland,19.717773,16.925455,8.906575,15.232747,20.057617,16.468750,9.600261,14.829427,...,20.564127,16.722006,9.213216,15.555989,20.917969,17.012045,10.346354,15.262370,21.477539,16.115561
15349,Zilzie,Queensland,22.587402,19.662760,11.595052,18.299154,22.340494,19.039389,12.831706,18.845703,...,23.673502,19.732422,11.961914,19.632486,23.763672,20.079428,12.621745,18.472330,24.057291,19.426107
15350,Zuccoli,Northern Territory,24.351562,23.412109,18.540365,24.099283,25.028971,22.665039,18.866861,23.985678,...,24.728842,22.765299,18.393881,23.942383,25.652018,23.565430,17.700521,22.738932,25.732422,23.575195
15351,Zumsteins,Victoria,12.348193,8.370638,3.242513,6.972786,11.995639,7.113542,4.226823,6.129167,...,11.962337,8.596581,2.950912,5.939583,12.079297,8.303906,3.780469,5.558529,11.525586,7.814550


In [ ]:
precip_gdf.to_csv("../../data/processed/processed_climate_data/suburb/mean_precip_mm_suburb.csv")
tmin_gdf.to_csv("../../data/processed/processed_climate_data/suburb/mean_tmin_suburb.csv")
tmax_gdf.to_csv("../../data/processed/processed_climate_data/suburb/mean_tmax_suburb.csv")

In [ ]:
import geopandas as gpd

test = gpd.read_file("../../data/raw/SAL_2021_AUST_GDA2020.shp")

In [ ]:
test

In [ ]:
precip_gdf.columns

In [ ]:
test.shape

In [ ]:
precip_gdf.crs

In [ ]:
climate_gdf = gpd.GeoDataFrame(
            precip,
            geometry=gpd.points_from_xy(precip.lon, precip.lat),
            crs="EPSG:7844"
        )

In [ ]:
climate_gdf.crs

In [ ]:
def aggregate_shape(df):
        climate_gdf = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(df.lon, df.lat),
            crs="EPSG:7844"
        )

        suburbs_gdf = gpd.read_file("../../data/raw/SAL_2021_AUST_GDA2020.shp")

        climate_gdf = climate_gdf.to_crs(suburbs_gdf.crs)

        joined = gpd.sjoin(climate_gdf, suburbs_gdf, how="left", predicate="intersects")
        aggregated = joined.groupby("SAL_NAME21")[df.columns.to_list()[2:]].mean().reset_index()

        aggregated = aggregated.merge(
            suburbs_gdf[["SAL_NAME21", "geometry"]],
            on = "SAL_NAME21",
            how = "left"
        )

        return aggregated

In [ ]:
aggregate_shape(precip)

In [ ]:
unmatched_suburbs = test[test["SAL_NAME21"].isin(precip_gdf["SAL_NAME21"].unique())]

In [ ]:
unmatched_suburbs["SHAPE_Area"].describe()

In [ ]:
tmin_gdf[tmin_gdf["SAL_NAME21"].str.contains("Mount Buller")][[i for i in tmin_gdf.columns if "2020" in i]]

In [ ]:
state_polygons

In [ ]:
import folium
import geopandas
import geopandas as gpd

merged = state_polygons.merge(tmax_gdf, on="SAL_NAME21", how="inner")

merged = gpd.GeoDataFrame(merged, geometry="geometry", crs=state_polygons.crs)

m = merged[merged["STE_NAME21"] == "Victoria"].explore(
    column="tmax_2020_January",
    cmap="coolwarm",
    tooltip=["SAL_NAME21", "STE_NAME21", "tmax_2020_January"],
    popup=False,
    tiles="OpenStreetMap"
)

In [ ]:
m.save("../../data/processed/tmax_test_vis.html")

In [ ]:
precip_gdf["STE_NAME21"].unique()

In [22]:
import folium
import geopandas
import geopandas as gpd
merged = hexagon_polygons.merge(precip_gdf_hex, on="hex_id", how="inner")

merged = gpd.GeoDataFrame(merged, geometry="geometry", crs=state_polygons.crs)

m = merged.explore(
    column="2020_month_1",
    cmap="Blues",
    tooltip=["2020_month_1"],
    popup=False,
    tiles="OpenStreetMap"
)

In [14]:
merged

,hex_id,geometry,2000_month_1,2000_month_2,2000_month_3,2000_month_4,2000_month_5,2000_month_6,2000_month_7,2000_month_8,...,2019_month_8,2019_month_9,2019_month_10,2019_month_11,2019_month_12,2020_month_1,2020_month_2,2020_month_3,2020_month_4,2020_month_5
0,8494181ffffffff,"POLYGON ((125.36694 -14.00928, 125.33034 -14.2...",867.088013,454.851349,358.260590,189.604156,0.000000,0.000000,0.000000,0.000000,...,0.115461,0.180681,2.815063,3.247158,34.392666,390.576996,259.592987,185.280472,21.990774,24.313442
1,8494183ffffffff,"POLYGON ((125.77817 -14.09717, 125.74132 -14.3...",687.659058,432.509460,395.888855,160.125275,0.000000,0.000000,0.070158,0.000000,...,0.060838,0.078605,9.381511,2.869449,49.375256,393.659546,279.375519,167.103607,16.364960,15.356702
2,8494185ffffffff,"POLYGON ((125.10709 -14.29543, 125.07068 -14.5...",900.685913,454.094025,367.768097,199.135223,0.000000,0.000000,0.000000,0.000000,...,0.035670,0.442845,5.989515,7.980640,54.658459,417.313171,283.767883,177.775223,31.924309,41.946873
3,8494187ffffffff,"POLYGON ((125.51733 -14.38384, 125.48067 -14.6...",857.526733,440.750488,357.980133,175.603088,0.000000,0.000000,0.000000,0.000000,...,0.029434,0.327491,9.453673,5.350500,61.285755,396.500061,245.987686,161.828979,11.209978,25.460663
4,849418bffffffff,"POLYGON ((125.62755 -13.72129, 125.59077 -13.9...",700.191284,443.565033,392.694153,161.558121,0.000000,0.000000,0.098701,0.000000,...,0.173725,0.068377,0.947865,0.707699,16.445742,371.919952,222.186813,169.182602,20.113178,13.147272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,84d896dffffffff,"POLYGON ((145.31644 -41.27815, 145.37661 -41.0...",47.247417,70.529465,55.779736,77.514175,193.512619,135.456161,231.210037,97.447937,...,157.070938,130.342163,57.633720,142.944504,84.965988,71.859993,58.097557,104.607155,191.255798,134.711609
4916,84d8b25ffffffff,"POLYGON ((145.53013 -42.84398, 145.59091 -42.5...",84.777550,93.623489,100.546051,158.684265,301.344757,155.779312,252.165359,132.487671,...,266.119080,166.122345,110.052032,193.423126,148.741959,107.379471,115.408005,191.338882,294.619629,201.118149
4917,84d8b2dffffffff,"POLYGON ((145.73197 -43.25484, 145.79246 -43.0...",74.088623,91.496407,107.901398,192.292206,336.918518,182.439896,209.128098,96.102470,...,299.467346,173.875732,125.216148,235.062683,191.782166,104.983322,120.708939,170.415909,287.167450,225.111145
4918,84d8b65ffffffff,"POLYGON ((146.52955 -43.74646, 146.58788 -43.5...",74.464180,77.826233,87.090607,121.538948,249.426270,138.588882,181.993469,103.638374,...,200.949097,113.459969,84.964790,185.752960,122.815750,81.240334,90.481949,150.449753,223.118759,129.603256


In [23]:
m.save("../../data/processed/precip_hex_4_test.html")

In [27]:
precip_gdf

,precip_2000_Summer,precip_2000_Autumn,precip_2000_Winter,precip_2000_Spring,precip_2001_Summer,precip_2001_Autumn,precip_2001_Winter,precip_2001_Spring,precip_2002_Summer,precip_2002_Autumn,...,precip_2018_Summer,precip_2018_Autumn,precip_2018_Winter,precip_2018_Spring,precip_2019_Summer,precip_2019_Autumn,precip_2019_Winter,precip_2019_Spring,precip_2020_Summer,precip_2020_Autumn
0,27.505859,59.016277,53.120770,99.526695,53.266277,22.575521,53.993652,46.343262,79.029625,29.483887,...,82.935875,16.992188,27.331869,46.511719,68.317543,48.586590,22.619629,23.737631,54.519531,78.860352
1,39.426659,56.620117,47.366077,103.685028,68.146873,36.193684,61.956646,53.333923,90.892120,29.944206,...,74.811852,15.280990,28.137695,43.355602,66.063675,56.852413,19.597265,30.557226,62.572788,63.422977
2,25.193359,59.354492,50.149738,97.101234,55.552082,23.125000,52.467121,48.043293,85.612953,26.597006,...,87.896484,16.075846,24.901367,44.889324,69.720055,48.420898,18.017904,23.445963,45.335938,79.037758
3,36.247234,69.807724,71.680450,126.391701,63.742294,29.053926,71.593102,57.818684,87.234047,37.826065,...,95.180016,22.277885,39.204208,53.165798,73.332138,65.097984,36.208439,31.752169,63.144421,96.850914
4,49.038887,78.246513,90.609467,146.002365,81.930901,38.852795,91.679199,74.283455,100.666229,44.323406,...,111.066658,28.750046,54.664875,61.548481,82.689308,80.490349,47.120117,41.945801,77.534340,108.591309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15348,65.989746,63.558594,27.524414,64.845703,113.352867,85.658852,21.579428,97.279945,64.656578,62.789387,...,129.026367,93.496742,23.684896,75.451500,41.973633,102.103516,28.097330,33.292969,234.041672,51.332684
15349,98.415527,82.163086,39.416668,139.104172,89.066734,37.811848,26.592447,41.794270,101.508789,36.476887,...,119.351562,47.907879,17.215820,41.034832,82.844727,65.255531,25.522461,20.674479,137.588547,116.366211
15350,477.617676,227.099930,0.245117,58.060547,350.787750,112.625977,5.316406,95.446289,283.611969,75.196617,...,422.092133,99.055664,0.000000,78.244789,196.517258,108.434570,0.005534,38.725586,282.543304,115.216469
15351,27.064062,42.144661,60.663055,73.819786,29.944532,34.352215,84.418587,84.527473,27.480795,25.432129,...,19.838087,40.529621,80.903389,30.939356,32.576660,44.897625,80.313896,38.003971,33.190037,58.304718
